In [1]:
import dgl
import torch

In [5]:
n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_max_clicks = 10
n_user_classes = 5

In [73]:
# 关注边
torch.manual_seed(8)
follow_src = torch.randint(0, n_users, (n_follows,))
follow_dst = torch.randint(0, n_users, (n_follows,))
# (follow_dst==follow_src).sum()

# 点击边
click_src = torch.randint(0, n_users, (n_clicks,))
click_dst = torch.randint(0, n_items, (n_clicks,))

# 不喜欢边
dislike_src = torch.randint(0, n_users, (n_dislikes,))
dislike_dst = torch.randint(0, n_items, (n_dislikes,))

In [246]:
hg = dgl.heterograph(
    {
        ("user", "follow", "user"): (follow_src, follow_dst),
        ("user", "followed-by", "user"): (follow_dst, follow_src),
        ("user", "click", "item"): (click_src, click_dst),
        ("item", "clicked-by", "user"): (click_dst, click_src),
        ("user", "dislike", "item"): (dislike_src, dislike_dst),
        ("item", "disliked-by", "user"): (dislike_dst, dislike_src),
    }
)
# 节点特征
hg.nodes["user"].data["feat"] = torch.randn(n_users, n_hetero_features)
hg.nodes["item"].data["feat"] = torch.randn(n_items, n_hetero_features)
hg.nodes["user"].data["label"] = torch.randint(0, n_user_classes, (n_users,))
hg.nodes["user"].data["train_mask"] = torch.ones(n_users).bernoulli(0.6)

# 边特征
hg.edges["click"].data["label"] = torch.randint(1, n_max_clicks, (n_clicks,))
hg.edges["click"].data["train_mask"] = torch.ones(n_clicks).bernoulli(0.6)